<center><span style="font-family:Comic Sans MS; color:navy "><h1>Regression with a Mohs Hardness Dataset(LGBM+CatBoost+XGB)</h1></span></center>

# <b>a <span style='color:#94D4F6'>|</span> Table of Conents</b>
<a id="toc"></a>
- [1. Import Libraries](#1)
- [2. Read Dataset](#2)
- [3. Preprocessing](#3)
- [4. Visualization](#4)
- [5. Feature Engineering](#5)
- [6. Split Dataset](#6)
- [7. Modeling](#7)
- [8. Predict test data](#8)
- [9. Evaluation](#9)
- [9. Submission](#10)



<a id="1"></a>
# <b>1 <span style='color:#94D4F6'>|</span> Import Libraries</b>

In [7]:
!pip install catboost

     -                                        3.8/101.0 MB 4.0 kB/s eta 6:42:32


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Hasibullah Aman\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 435, in _error_catcher
    yield
  File "C:\Users\Hasibullah Aman\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 516, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "C:\Users\Hasibullah Aman\anaconda3\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "C:\Users\Hasibullah Aman\anaconda3\lib\http\client.py", line 463, in read
    n = self.readinto(b)
  File "C:\Users\Hasibullah Aman\anaconda3\lib\http\client.py", line 507, in readinto
    n = self.fp.readinto(b)
  File "C:\Users\Hasibullah Aman\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\Hasibullah Aman\anaconda3\lib\ssl.py", line 1242, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Users\Hasibullah Aman\anacond

In [6]:
import numpy as np # for doing some statical operation
import pandas as pd # for working with data frame

# plot
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# encoding categorical features
from sklearn.preprocessing import(
    StandardScaler,# standard scalar will scale data from (-3 to 3)
    LabelEncoder, # for label encoding categorical features
    OneHotEncoder, # used for onehot encoding categorical features
) 
# split the dataset
from sklearn.model_selection import train_test_split

# Model
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Evaluation
from sklearn.metrics import median_absolute_error

ModuleNotFoundError: No module named 'catboost'

<a id="2"></a>
# <b>2 <span style='color:#94D4F6'>|</span> Read Dataset</b>

In [ ]:
df = pd.read_csv('/kaggle/input/playground-series-s3e25/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e25/test.csv')
sub = pd.read_csv('/kaggle/input/playground-series-s3e25/sample_submission.csv')

In [ ]:
test.head()

<a id="3"></a>
# <b>3 <span style='color:#94D4F6'>|</span> PreProcessing</b>

In [ ]:
df.shape

In [ ]:
df.columns

### Types of columns

In [ ]:
df.dtypes

In [ ]:
df.info()

### Describe the dataset

In [ ]:
df.describe().style.background_gradient(cmap ='summer')

In [ ]:
# Show missing Value
msno.matrix(df)

<span style='font-family:comic Sans MS; color:navy'>As you can see, we don't have any missing value now in our dataSet</span>

<a id="4"></a>
# <b>4 <span style='color:#94D4F6'>|</span> Exploratory Data Analysis</b>

In [ ]:
sns.pairplot(df)
plt.show()

<a id="5"></a>
# <b>5 <span style='color:#94D4F6'>|</span> Feature Engineering</b>

In [ ]:
# correlation matrix between numerical cols in the dataset.
fig, ax = plt.subplots(figsize=(10,18))
coer = df.corr()
sns.heatmap(coer, annot =True, ax=ax, vmin=-1, vmax=1, center= 0, cmap= 'coolwarm', linewidths=3, linecolor='black')
plt.show()

<span style='font-family:comic Sans MS; color:navy'>As you can see, we have some unnecessary columns which we didn't need for Prediction or maybe have a weak correlation.</span>

In [ ]:
df.corr()['Hardness'].sort_values(ascending=False) 

In [ ]:
df = df.drop(['id'],axis=1)
test = test.drop(['id'],axis=1)

<div style="color:white;display:fill;border-radius:8px;background-color:#03112A;font-size:150%;letter-spacing:1.0px"><p style="padding: 10px;color:white;"><b><b><span style='color:#94D4F6'>5.1 |</span></b> Feature Scaling </b></p></div>

In [ ]:
# # scalar for all numerical columns excluding target column
# sc_X = StandardScaler()
# # scalar for the target column
# sc_y = StandardScaler()
# # list of col  names that include numerical values
# numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
# # list of all numeric cols excluding the target col
# numeric_cols_x = [x for i,x in enumerate(numeric_cols) if(x!='Hardness')]
# # list includes target col name
# numeric_cols_y = [x for i,x in enumerate(numeric_cols) if(x not in numeric_cols_x)]
# df[numeric_cols_x] = sc_X.fit_transform(df[numeric_cols_x])
# df[numeric_cols_y]= sc_y.fit_transform(df[numeric_cols_y])
# df.head()

<a id="6"></a>
<div style="color:white;display:fill;border-radius:8px;background-color:#03112A;font-size:150%;letter-spacing:1.0px"><p style="padding: 10px;color:white;"><b><b><span style='color:#94D4F6'>5.2 |</span></b> Spliting The Dataset to Train and Test</b></p></div>

In [ ]:
X = df.drop('Hardness', axis=1)
y = df['Hardness']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train

In [ ]:
y_train

<a id="7"></a>
# <b>6 <span style='color:#94D4F6'>|</span> Modeling</b>

In [ ]:
cat_model = CatBoostRegressor(random_state=42, silent=True)
xgb_model = XGBRegressor(random_state=42)
lgb_model = LGBMRegressor(random_state=42)

In [ ]:
# Train models
cat_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)
lgb_model.fit(X_train, y_train)

<a id="8"></a>
# <b>7 <span style='color:#94D4F6'>|</span> Predictions</b>

In [ ]:
cat_predictions = cat_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)
lgb_predictions = lgb_model.predict(X_test)

In [ ]:
# Ensemble predictions (simple averaging)
ensemble_predictions = (cat_predictions + xgb_predictions + lgb_predictions) / 3

<a id="9"></a>
# <b>8 <span style='color:#94D4F6'>|</span> Evaluation</b>

In [ ]:
# Evaluate ensemble model
ensemble_medae = median_absolute_error(y_test, ensemble_predictions)
print(f"Ensemble Model MedAE: {ensemble_medae}")

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter Plot: Predicted vs Actual
axes[0].scatter(y_test, ensemble_predictions, color='blue', alpha=0.5)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
axes[0].set_xlabel('Actual Values')
axes[0].set_ylabel('Predicted Values')
axes[0].set_title('Scatter Plot: Predicted vs. Actual')

# Line Plot: Predicted vs Actual
axes[1].plot(y_test.values, color='blue', alpha=0.5, label='Actual')
axes[1].plot(ensemble_predictions, color='green', alpha=0.5, label='Predicted')
axes[1].set_title('Line Plot: Predicted vs. Actual')
axes[1].legend()

plt.tight_layout()
plt.show()

<a id="10"></a>
# <b>9 <span style='color:#94D4F6'>|</span> Submission</b>

In [ ]:
# predict
test_1 = pd.read_csv('/kaggle/input/playground-series-s3e25/test.csv')
test = test_1.drop('id',axis = 1)

In [ ]:
# ------- predict
cat_prediction = cat_model.predict(test)
xgb_prediction = xgb_model.predict(test)
lgb_prediction = lgb_model.predict(test)
# Ensemble predictions (simple averaging)
ensemble_predictions = (cat_prediction + xgb_prediction + lgb_prediction) / 3
submission = pd.DataFrame({
    "id": test_1.id,
    "Hardness": ensemble_predictions
})
submission

In [ ]:
submission.to_csv('./submission.csv', index=False)

<center>
<span style="font-family:Comic Sans MS; color:red"><h2>Thank You😊</h2></span>
</center>